## Day1
https://adventofcode.com/2021/day/1

In [ ]:
with open('inputs_2021/day1.txt', 'r') as file:
    depths = [int(row.strip()) for row in file]

measurement_increases = sum(
    current > previous for previous, current in zip(depths, depths[1:])
)

print(measurement_increases)

1711


In [22]:
sliding_windows = [
    sum(depths[i:i+3]) for i in range(len(depths) - 2)
]

sliding_windows_increaces = sum(
    current > previous for previous, current in zip(sliding_windows, sliding_windows[1:])
)
print(sliding_windows_increaces)

1743


## Day2
https://adventofcode.com/2021/day/2

In [38]:
position_changes = {}

with open('inputs_2021/day2.txt', 'r') as file:
    for row in file:
        key, value = row.strip().split()
        position_changes[key] = position_changes.get(key, 0) + int(value)

print(position_changes['forward'] * (position_changes['down'] - position_changes['up']))

2187380


In [ ]:
from collections import defaultdict

positions = defaultdict(int)

with open('inputs_2021/day2.txt', 'r') as file:
    for row in file:
        key, value = row.strip().split()
        value = int(value)
        aim = 0

        match key:
            case 'down':
                aim += value
            case 'up':
                aim -= value
            case  'forward':
                positions['forward'] += value
                positions['down'] += aim * value

print(positions)
print(positions['forward'] * positions['down'])

defaultdict(<class 'int'>, {'forward': 1790, 'down': 1165563, 'aim': 1222})
2086357770


## Day3
https://adventofcode.com/2021/day/3

In [116]:
from collections import defaultdict

positions = defaultdict(int)

with open('inputs_2021/day3.txt', 'r') as file:
    file_len = 0
    for line in file:
        file_len += 1
        for i, bit in enumerate(line.strip()):
            positions[i] += int(bit)

threshold = file_len / 2

gamma_rate, epsilon_rate = zip(*[
    ('1', '0') if val >= threshold else ('0', '1') 
    for val in positions.values()
])

print(int(''.join(gamma_rate), 2) * int(''.join(epsilon_rate), 2))

693486


In [ ]:
def parse_input(filepath: str) -> list[list[int]]:
    """Reads the input file and returns a list of lists of integers."""
    with open(filepath, 'r') as file:
        return [list(map(int, line.strip())) for line in file]

def find_rating(data: list[list[int]], is_gamma: bool) -> int:
    bit_length = len(data[0])

    for i in range(bit_length):
        if len(data) == 1:
            break
        ones = sum(row[i] for row in data)
        zeroes = len(data) - ones
        if is_gamma:
            keep_bit = 1 if ones >= zeroes else 0
        else:
            keep_bit = 0 if ones >= zeroes else 1
        data = [row for row in data if row[i] == keep_bit]

    return int(''.join(map(str, data[0])), 2)

data = parse_input('inputs_2021/day3.txt')

oxygen_rating = find_rating(data, True)
co2_rating = find_rating(data, False)

# Multiply and print
print(oxygen_rating * co2_rating)

3379326


## Day4
https://adventofcode.com/2021/day/4

In [124]:
def parse_input(filepath: str) -> list[list[int]]:
    """Reads the input file and returns a list of lists of integers."""
    with open(filepath) as f:
        parts = f.read().strip().split('\n\n')

    print(parts)
    inputs = list(map(int, parts[0].split(',')))

    boards = [
        [list(map(int, row.split())) for row in block.splitlines()]
        for block in parts[1:]
    ]
    return inputs, boards
    

inputs, boards_raw = parse_input('inputs_2021/day4.txt')


['79,9,13,43,53,51,40,47,56,27,0,14,33,60,61,36,72,48,83,42,10,86,41,75,16,80,15,93,95,45,68,96,84,11,85,63,18,31,35,74,71,91,39,88,55,6,21,12,58,29,69,37,44,98,89,78,17,64,59,76,54,30,65,82,28,50,32,77,66,24,1,70,92,23,8,49,38,73,94,26,22,34,97,25,87,19,57,7,2,3,46,67,90,62,20,5,52,99,81,4', ' 7 42 22 92 60\n 8 88 99 13 12\n16 62 86 24 77\n20 57 19 67 46\n36 83 54 63 82', ' 7 86 50 78 16\n83 45 67 94 58\n21 98 99 85 43\n71 19 31 22  4\n70 51 34 11 61', ' 4 95 84 51 36\n43 40 37 23 85\n14 90  8 59 99\n 0 88 68 93 81\n25  6 55 19 48', '15 39 78  6 13\n71  3 81 95 62\n22 46 67 72 40\n89 69  0 37 41\n68 79 58 16 42', '63 50 77 34 12\n29 42 20 17 47\n80 10 30 72 66\n 5 89 64 25 21\n91 88 45 44 37', '78 89 32 26 56\n 8 40 54 25 49\n36 30 21 23  3\n12 58  2 29  7\n33 99 15 84 44', '96 68 56 49 43\n55 22 16 91 32\n 2 17 61 12 37\n25 72  1 31 88\n57 34 42  8 71', '18 39 86 94 60\n96 85 64 51 28\n48 14 23 36 35\n 6 84 99 90 81\n43 41 74 68 32', ' 9 58 60  7 61\n96 33 67  0 19\n77  2 14 99 79\n1

In [ ]:
import numpy as np

class BingoBoard:
    def __init__(self, grid):
        self.numbers = np.array(grid)
        self.marked = np.zeros((5, 5), dtype=bool)

    def mark(self, number):
        self.marked |= (self.numbers == number)

    @property
    def is_winning(self):
        return np.any(np.all(self.marked, axis=0)) or np.any(np.all(self.marked, axis=1))

    @property
    def unmarked_sum(self):
        return np.sum(self.numbers[~self.marked])

    def score(self, last_number):
        return self.unmarked_sum * last_number

boards = [BingoBoard(grid) for grid in boards_raw]

for number in inputs:
    for board in boards:
        board.mark(number)
        if board.is_winning:
            print("Winner!")
            print("Score:", board.score(number))
            break
    else:
        continue
    break

Winner!
Score: 97565


In [ ]:
def find_last_winning_board(boards_raw, inputs) -> int:

    remaining_boards = [BingoBoard(grid) for grid in boards_raw]

    for number in inputs:
        newly_won = []

        for board in remaining_boards:
            board.mark(number)
            if board.is_winning:
                if len(remaining_boards) == 1:
                    print("Looser found!")
                    return board.score(number)
                else:
                    newly_won.append(board)

        remaining_boards = [b for b in remaining_boards if b not in newly_won]

print(f"Looser's score is {find_last_winning_board(boards_raw, inputs)}")

Looser found!
Looser's score is 1827


## DAY5

In [5]:
import numpy as np
import matplotlib.pyplot as plt


def parse_lines(filename):
    with open(filename) as f:
        for line in f:
            x1, y1, x2, y2 = map(int, line.replace(' -> ', ',').split(','))
            yield x1, y1, x2, y2

def get_grid(lines, diagonals=False, size=1000):
    grid = np.zeros((size, size), dtype=int)
    for x1, y1, x2, y2 in lines:
        dx = np.sign(x2 - x1)
        dy = np.sign(y2 - y1)

        length = max(abs(x2 - x1), abs(y2 - y1))
        if not diagonals and dx != 0 and dy != 0:
            continue

        for i in range(length + 1):
            grid[y1 + i * dy, x1 + i * dx] += 1
    return grid


In [6]:
lines = list(parse_lines('inputs_2021/day5.txt'))
grid1 = get_grid(lines, diagonals=False)
grid2 = get_grid(lines, diagonals=True)

print("Part 1:", np.sum(grid1 >= 2))
print("Part 2:", np.sum(grid2 >= 2))

Part 1: 5092
Part 2: 20484


## DAY 6

In [1]:
from collections import Counter

# Parse input
with open("inputs_2021/day6.txt") as f:
    lanternfishes = list(map(int, f.readline().strip().split(",")))

def fish_simulation(fish_counts: dict, days_to_simulate: int):

    fish_dict =  fish_counts.copy()
    for _ in range(days_to_simulate):
        next_day = Counter()
        for timer, count in fish_dict.items():
            if timer == 0:
                next_day[6] += count
                next_day[8] += count
            else:
                next_day[timer - 1] += count
        fish_dict = next_day

    # Final count
    return sum(fish_dict.values())


In [2]:
fish_counts = Counter(lanternfishes)

fish_after_80_days = fish_simulation(fish_counts, 80)
fish_after_256_days = fish_simulation(fish_counts, 256)


print(fish_after_80_days)
print(fish_after_256_days)

345387
1574445493136
